# Influence of distortions on the information signal

This demo is supposed to show the influence of a nonideal transmission or [communication channel](https://en.wikipedia.org/wiki/Communication_channel) on a [baseband information signal](https://en.wikipedia.org/wiki/Baseband).

This demo is written by [Markus Nölle](https://www.htw-berlin.de/hochschule/personen/person/?eid=9586) for a basic course on [communications engineering](https://en.wikipedia.org/wiki/Telecommunications_engineering) hold at the [university of applied sciences, Berlin](https://www.htw-berlin.de/).

## Import libraries and set defaults

In [1]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import ipywidgets as widgets

%config InlineBackend.figure_format = 'svg'

plt.style.use('noelle.mplstyle')
plt.rcParams["figure.figsize"] = (4.0,2.5)

#inline, widget 
%matplotlib inline

## General parameters
Basic parameters like sample rate, time axis and frequency axis are defined.

In [2]:
n_bits = 100
sps = 10
sample_rate = 1

t = np.arange(0, sps*sample_rate*n_bits, 1/sample_rate)
f = np.fft.fftshift(np.fft.fftfreq(np.size(t), 1/sample_rate))
n = np.size(t) # length of signal

## Communication channel

We assume the communication channel to be a [linear timeinvariant (LTI) system](https://en.wikipedia.org/wiki/Linear_time-invariant_system). So it can simply be characterized by its [impulse response](https://en.wikipedia.org/wiki/Linear_time-invariant_system#Impulse_response_and_convolution) or its [frequency response](https://en.wikipedia.org/wiki/Linear_time-invariant_system#Exponentials_as_eigenfunctions).

A distortion-free communication channel only performs a scaling (with a factor $s$) and a potential delay (of the time $\tau$) on the input signal. This distortion-free channel can therefore be described with following impulse response and frequency responste
$$
\begin{eqnarray}
h(t) &=&  s \cdot \delta(t-\tau)\\
H(j\omega) &=& s \cdot \exp(-j\omega\tau),
\end{eqnarray}
$$
respectively.

When introducing deviations from this distotion-free system, the input signal will be distorted by the communiction channel and hence, the signal recovery might be hindered. Two simple, parametrizable deviations from the distortion-free system are exemplary shown here.

### Cosine ripple on the amplitude transfer function

The amplitude transfer function will distorted by an additional, unwanted cosine ripple which can be described as
$$
H(j\omega) = (s + a \cdot \cos(\alpha \omega)) \cdot \exp(-j\omega\tau),
$$
while $a$ is the strength (amplitude) of this distortion and $\alpha$ characterized the frequency of the cosine ripple. This distortion leads to the follwoing impulse response
$$
h(t) =  s \cdot \delta(t-\tau) + \frac{a}{2}(\delta(t-\alpha-\tau)+\delta(t+\alpha-\tau)).
$$
Convolution of the input signal ($x(t)$) with this inpulse response reveals the distorted output signal
$$
\begin{eqnarray}
y(t) &=&  x(t) \ast h(t)\\
&=& s\cdot x(t-\tau) + \frac{a}{2}(x(t-\tau-\alpha) + x(t-\tau+\alpha)).
\end{eqnarray}
$$
It can be seen, that this simple distortion causes symmetric, scaled echos of the input signal after transmission. The strength of these echos is dependent on the amplitude of the ripple, while their distance to the main output is dependent on the ripple's frequency. It becomes apparent that this distortion will eventually cause interference to temporally neighbouring bits and therefore reduce the signal quality.

### Sine ripple on the phase transfer function

## Generate information signal

In [3]:
def plot_dist_signal(a=0.0, alpha=1.0, b=0.0, beta=1.0):
    # generate whole information signal
    rg = np.random.default_rng(12345)
    sig = rg.integers(0, high=2, size=n_bits)
    sig = np.repeat(sig, sps)
    transfer_function = signal.windows.gaussian(sig.size, 100)
    sig = np.real(np.fft.ifft(np.fft.fft(sig) * np.fft.fftshift(transfer_function)))

    sig_one = np.zeros(n_bits)
    sig_one[2] = 1.0
    sig_one = np.repeat(sig_one, sps)
    sig_one = np.real(np.fft.ifft(np.fft.fft(sig_one) * np.fft.fftshift(transfer_function)))

    omega = 2*np.pi*f
    tau = 1;
    H = (np.ones(sig.size) + a * np.cos(alpha * omega)) * np.exp(-1j * (omega * tau - b * np.sin(beta * omega)))

    sig_ch = np.real(np.fft.ifft(np.fft.fft(sig) * np.fft.fftshift(H)))
    sig_ch = sig_ch / np.max(sig_ch)
    sig_one_ch = np.real(np.fft.ifft(np.fft.fft(sig_one) * np.fft.fftshift(H)))
    sig_one_ch = sig_one_ch / np.max(sig_one_ch)

    # plotting
    n_row = 2
    n_col = 4
    fig_size = [i*j for i,j in zip(plt.rcParams['figure.figsize'], [n_col, n_row])]
    fig = plt.figure(figsize=fig_size)

    ax = fig.add_subplot(1, 4, 1)
    plt.plot(omega, np.abs(H), 'C0', omega, np.angle(H)/np.pi, 'C3')
    ax.set(xlabel=r"$\Omega$ / rad", ylabel="amplitude (a.u.) / phase (rad/$\pi$)", title="transfer function")
    ax.legend(('$|H(j\Omega)|$','arg($H(j\Omega)$)'))
    ax.set_ylim((-1,2))

    ax = fig.add_subplot(n_row, n_col, 2)
    plt.plot(t[0:int(5*sps):], sig_one[0:int(5*sps):] ,'C0', t[0:int(5*sps):], sig_one_ch[0:int(5*sps):] ,'C1')
    ax.set(xlabel="time / s", ylabel="amplitude / a.u.", title="single '1'-pulse")
    ax.legend(('input signal','output signal'))

    ax = fig.add_subplot(n_row, n_col, 3)
    n_bits_plot = 10
    plt.plot(t[0:int(n_bits_plot*sps):], sig[0:int(n_bits_plot*sps):] ,'C0', t[0:int(n_bits_plot*sps):], sig_ch[0:int(n_bits_plot*sps):] ,'C1')
    ax.set(xlabel="time / s", ylabel="amplitude / a.u.", title="bit pattern")
    ax.legend(('input signal','output signal'))
    
    ax = fig.add_subplot(1, 4, 4)
    Sig = np.abs(np.fft.fftshift(np.fft.fft(sig)))
    Sig = 20*np.log10(Sig/np.max(Sig))
    Sig_ch = np.abs(np.fft.fftshift(np.fft.fft(sig_ch)))
    Sig_ch = 20*np.log10(Sig_ch/np.max(Sig_ch))
    plt.plot(omega, Sig ,'C0', omega, Sig_ch ,'C1')
    ax.set(xlabel=r"$\Omega$ / rad", ylabel="norm. amplitude / dB", title="signal amplitude spectra")
    ax.legend(('input signal','output signal'))
    ax.set_ylim((-50,0))


    ax = fig.add_subplot(n_row, n_col, 6)
    plt.plot(t[0:int(2*sps):], np.reshape(np.roll(sig, int(sps/2)),(int(2*sps),-1),order='F') ,'C0')
    ax.set(xlabel="time / s", ylabel="amplitude / a.u.", title="input signal")

    ax = fig.add_subplot(n_row, n_col, 7)
    plt.plot(t[0:int(2*sps):], np.reshape(np.roll(sig_ch, int(sps/2)),(int(2*sps),-1),order='F') ,'C1')
    ax.set(xlabel="time / s", ylabel="amplitude / a.u.", title="output signal")


    plt.tight_layout()

In [4]:
w_a = widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=0.0, continuous_update=False, description=r'a')
w_alpha = widgets.FloatSlider(min=0.0, max=10.0, step=0.1, value=1.0, continuous_update=False, description=r'$\alpha$')
w_b = widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=0.0, continuous_update=False, description=r'b')
w_beta = widgets.FloatSlider(min=0.0, max=10.0, step=0.1, value=1.0, continuous_update=False, description=r'$\beta$')

ui = widgets.HBox([w_a, w_alpha, w_b, w_beta])

out = widgets.interactive_output(plot_dist_signal, {'a':w_a, 'alpha':w_alpha, 'b':w_b, 'beta':w_beta})
out.layout.height = '500px'

display(ui, out)

Output(layout=Layout(height='500px'))